In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('CHEMIN', index_col='id')
X,y = train.drop('TARGET',axis=1),train['TARGET']

X.head()

# Data Analysis

In [ ]:
train.isna().sum()

NameError: name 'train' is not defined

In [3]:
features = list(X.columns)
num_cols = list(X.select_dtypes(exclude='object').columns)
cat_cols = list(X.select_dtypes(include='object').columns)

NameError: name 'X' is not defined

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

class CategoricalVisualizer:
    def __init__(self, palette="Set2", figsize_unit=(6, 5)):
        self.palette = palette
        self.figsize_unit = figsize_unit

    def plot_counts(self, df, columns, title='Distribution of Categorical Variables'):
        n_cols = len(columns)
        n_rows = math.ceil(n_cols / 3)
        
        fig, axes = plt.subplots(
            n_rows, 3, 
            figsize=(self.figsize_unit[0] * 3, self.figsize_unit[1] * n_rows)
        )
        fig.suptitle(title, fontsize=16)
        axes = axes.flatten()

        for i, col in enumerate(columns):
            # Tri par fréquence pour une meilleure lecture
            order = df[col].value_counts().index
            
            sns.countplot(
                data=df, x=col, ax=axes[i], 
                palette=self.palette, order=order
            )
            
            axes[i].set_title(f'{col.replace("_", " ").title()}')
            axes[i].set_xlabel('')
            axes[i].tick_params(axis='x', rotation=45)

        # Supprimer les axes vides si le nombre de colonnes n'est pas un multiple de 3
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()
    
    def plot_boxplots(self, df, columns, target, title='Categorical Features vs Target'):
        """Affiche la relation entre catégories et une variable numérique cible."""
        n_cols = len(columns)
        n_rows = math.ceil(n_cols / 2) # 2 par ligne pour plus de clarté sur les boxplots
        
        fig, axes = plt.subplots(n_rows, 2, figsize=(18, n_rows * 6))
        fig.suptitle(title, fontsize=16, fontweight='bold')
        axes = axes.flatten()

        for i, col in enumerate(columns):
            # On trie les boîtes par la médiane pour mieux voir la tendance
            order = df.groupby(col)[target].median().sort_values().index
            
            sns.boxplot(
                data=df, x=col, y=target, ax=axes[i], 
                palette=self.palette, order=order,
                showmeans=True, # Ajoute un point pour la moyenne
                meanprops={"marker":"o", "markerfacecolor":"white", "markeredgecolor":"black"}
            )
            
            axes[i].set_title(f'{col.replace("_", " ").title()} vs {target.title()}')
            axes[i].set_xlabel('')
            axes[i].tick_params(axis='x', rotation=45)

        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()



class NumericalVisualizer:
    def __init__(self, palette="viridis", bins=30):
        self.palette = sns.color_palette(palette)
        self.bins = bins

    def plot_distributions(self, df, columns, title='Distribution of Numerical Variables'):
        n_cols = len(columns)
        n_rows = math.ceil(n_cols / 2) # On met souvent 2 colonnes pour les numériques (plus larges)
        
        fig, axes = plt.subplots(n_rows, 2, figsize=(18, n_rows * 5))
        fig.suptitle(title, fontsize=18, fontweight='bold')
        axes = axes.flatten()

        for i, col in enumerate(columns):
            # Sélection de la couleur dans la palette
            color = self.palette[i % len(self.palette)]
            
            # Création de l'histogramme avec KDE (courbe de densité)
            sns.histplot(
                data=df, x=col, ax=axes[i], 
                kde=True, color=color, bins=self.bins
            )
            
            axes[i].set_title(f'Distribution: {col.replace("_", " ").title()}', fontsize=14)
            axes[i].set_xlabel('')
            axes[i].grid(axis='y', linestyle='--', alpha=0.7)

        # Nettoyage des axes vides
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout(rect=[0, 0.03, 1, 0.97])
        plt.show()
    
    def plot_regressions(self, df, columns, target, n_samples=500, title='Numerical Features vs Target'):
        """Affiche la relation linéaire entre les variables numériques et la cible."""
        # On limite le nombre de samples pour la lisibilité et la performance
        df_sample = df.sample(n=min(n_samples, len(df)), random_state=42)
        
        n_cols = len(columns)
        n_rows = math.ceil(n_cols / 2)
        
        fig, axes = plt.subplots(n_rows, 2, figsize=(18, n_rows * 6))
        fig.suptitle(title, fontsize=16, fontweight='bold')
        axes = axes.flatten()

        for i, col in enumerate(columns):
            color = self.palette[i % len(self.palette)]
            
            # regplot trace les points ET la droite de régression avec intervalle de confiance
            sns.regplot(
                data=df_sample, x=col, y=target, ax=axes[i],
                scatter_kws={'alpha':0.5, 's':30}, # Transparence des points
                line_kws={'color': 'red', 'lw': 2}, # Ligne de régression en rouge
                color=color
            )
            
            # Calcul de la corrélation pour l'afficher dans le titre
            correlation = df[col].corr(df[target])
            axes[i].set_title(f'{col.title()} vs {target.title()} (Corr: {correlation:.2f})')
            axes[i].set_xlabel(col.replace("_", " ").title())
            axes[i].set_ylabel(target.title())

        # Nettoyage des axes vides
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()
    
    def plot_heatmap(self, df, columns, target, title='Correlation Heatmap'):
        """Affiche la matrice de corrélation entre les variables numériques et la cible."""
        # On regroupe les variables et la cible
        cols_to_corr = columns + [target]
        corr_matrix = df[cols_to_corr].corr()

        plt.figure(figsize=(12, 10))
        
        # Masque pour ne voir que la partie inférieure (évite la répétition en miroir)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

        sns.heatmap(
            corr_matrix, 
            mask=mask,
            annot=True,           # Affiche les valeurs
            fmt=".2f",            # 2 décimales
            cmap='coolwarm',      # Bleu (négatif) à Rouge (positif)
            center=0,             # Le blanc représente l'absence de corrélation
            linewidths=.5, 
            cbar_kws={"shrink": .8}
        )

        plt.title(title, fontsize=16, fontweight='bold', pad=20)
        plt.xticks(rotation=45, ha='right')
        plt.show()

In [ ]:
# pour l'appeler :

#CATEGORICAL
viz = CategoricalVisualizer   
viz.plot_counts(train, cat_cols)

#NUMERICAL
viz2 = NumericalVisualizer()
viz2.plot_distributions(train, num_cols)